In [52]:
import pandas as pd
import os
import glob
import albumentations as A
import cv2
import numpy as np
from PIL import Image

In [53]:
cv2.__version__
A.__version__

'1.3.0'

In [58]:
root = '/opt/ml/input/data'

filenames = ['incorrect_mask', 'mask1', 'mask2', 'mask3', 'mask4', 'mask5', 'normal']

paths = []
df = pd.read_csv(os.path.join(root, 'train', 'train.csv'))
for path in df['path']:
    p = [glob.glob(os.path.join(root, 'train', 'images', path, f+'*')) for f in filenames]
    paths.extend(p)

In [59]:
Horizontal = A.Compose([
    A.HorizontalFlip(p=1)
])

Blur = A.Compose([
    A.MedianBlur(blur_limit=5, p=1)
])

Rotation = A.Compose([
    A.Rotate(limit=60, p=1, always_apply=True)
])

In [66]:
for p in paths:
    image = np.array(Image.open(p[0]))
    
    _Horizontal = Horizontal(image=image)
    _Blur = Blur(image=image)
    _Rotation = Rotation(image=image)
    _, ext = p[0].split('.')
    new_path = p[0].replace('train', 'off_aug')
    
    os.makedirs(os.path.dirname(new_path), exist_ok=True)
    
    Image.fromarray(Horizontal(image=image)["image"]).save(new_path[:-4] + '_H.' + ext)
    Image.fromarray(Blur(image=image)["image"]).save(new_path[:-4] + '_B.' + ext)
    Image.fromarray(Rotation(image=image)["image"]).save(new_path[:-4] + '_R.' + ext)